In [1]:
import os, sys
import random
import math
import json
import functools
import itertools
import importlib
from collections import defaultdict
from google.protobuf.json_format import MessageToDict

import pathlib
solitaire_path = pathlib.Path('../../../solitaire/')

if solitaire_path not in sys.path:
    sys.path.append(str(solitaire_path))
    
from solitaire_core.game import *
from solitaire_core import game, text_renderer
importlib.reload(game)
importlib.reload(text_renderer)

<module 'solitaire_core.text_renderer' from '../../../solitaire/solitaire_core/text_renderer.py'>

In [7]:
states = []

importlib.reload(text_renderer)
importlib.reload(game)
g = game.deal_game(is_random=False)

new_gs = VisibleGameState()
new_gs.CopyFrom(g.gs)
states.append(new_gs)

# g.apply_action(game.Action(type=TO_SS_S, suit=CLUBS))

# for _ in range(7):
#     g.apply_action(game.Action(type=TO_SS_S, suit=game.CLUBS))

# for _ in range(3):
#     g.apply_action(game.Action(type=game.TALON_TO_BUILD_STACK_NUM, suit=game.HEARTS, build_stack_dest=6))
#     g.apply_action(game.Action(type=game.TALON_TO_BUILD_STACK_NUM, suit=game.CLUBS, build_stack_dest=6))

# g.apply_action(game.Action(type=game.TALON_TO_BUILD_STACK_NUM, suit=game.HEARTS, build_stack_dest=6))

# for r in game.CardRank.values()[1:]:
#     g.gs.build_stacks[6] |= game.get_bitmask(game.CLUBS, r)
# g.gs.build_stacks[6] |= game.SUIT_MASK[game.CLUBS]

# for _ in range(6):
#     g.apply_action(
#         game.Action(type=game.TO_SUIT_STACK, suit=game.CLUBS)
#     )
    
# g.apply_action(
#     game.Action(type=game.SUIT_STACK_TO_BUILD_STACK_NUM, suit=game.CLUBS, build_stack_dest=5)
# )

# g.apply_action(game.Action(type=game.BUILD_STACK_NUM_TO_BUILD_STACK_NUM, build_stack_src=6, build_stack_dest=4))

# g.apply_action(game.Action(type=game.TALON_TO_BUILD_STACK_NUM, suit=game.SPADES, build_stack_dest=5))
# g.apply_action(game.Action(type=game.TALON_TO_BUILD_STACK_NUM, suit=game.HEARTS, build_stack_dest=5))
# g.apply_action(game.Action(type=game.TALON_TO_BUILD_STACK_NUM, suit=game.CLUBS, build_stack_dest=5))

# g.apply_action(game.Action(type=game.BUILD_STACK_NUM_TO_BUILD_STACK_NUM, build_stack_src=5, build_stack_dest=6))

for r in game.CardRank.values()[1:]:
    for s in game.Suit.values()[1:]:
        g.apply_action(game.Action(type=game.TO_SS_S, suit=s))
        
        new_gs = VisibleGameState()
        new_gs.CopyFrom(g.gs)
        states.append(new_gs)
        
for r in game.CardRank.values()[1:-1]:
    if r % 2 == 0:
        for s, dest in [(CLUBS, 1), (DIAMONDS, 2), (HEARTS, 3), (SPADES, 4)]:
            g.apply_action(Action(type=SS_S_TO_BS_N, suit=s, build_stack_dest=dest))
            
            new_gs = VisibleGameState()
            new_gs.CopyFrom(g.gs)
            states.append(new_gs)
    else:
        for s, dest in [(CLUBS, 2), (DIAMONDS, 1), (HEARTS, 4), (SPADES, 3)]:
            g.apply_action(Action(type=SS_S_TO_BS_N, suit=s, build_stack_dest=dest))

            new_gs = VisibleGameState()
            new_gs.CopyFrom(g.gs)
            states.append(new_gs)

# print(game._try_apply_action(g.gs, g.hgs, Action(type=SS_S_TO_BS_N, suit=CLUBS, build_stack_dest=2)))
            
# g.apply_action(game.Action(type=game.BUILD_STACK_NUM_TO_BUILD_STACK_NUM, build_stack_src=6, build_stack_dest=0))

# Modify to test stuff out:
# g.current_state.build_stacks[0] |= 1 << 5
# g.current_state.talon &= game.SUIT_MASK[game.HEARTS]
print(game.is_valid_game_state(g.gs))

# print(game.game_state_id_b85(g.gs), len(game.game_state_id_b85(g.gs)))
# print(game.get_state_id())

print(text_renderer.render(g.gs, g.get_valid_actions(False)))
# print("\n".join(json.dumps(list(MessageToDict(a).values())) for a in g.get_valid_actions()))

True
+------------------------------------------------------------------------+     Actions
|                                                                        |
|                                                                        |  0: TO_SS_S          CLUBS       
|                                                                        |  1: TO_SS_S          DIAMONDS    
|                                                                        |  2: TO_SS_S          HEARTS      
|                                                                        |  3: TO_SS_S          SPADES      
|                                                                        |
|             0        1        2        3        4        5        6    |  4: SS_S_TO_BS_N     DIAMONDS  1 
| +-----+           +-----+  +-----+  +-----+  +-----+                   |  5: SS_S_TO_BS_N     HEARTS    1 
| | A ♣ |           | K ♦ |  | K ♣ |  | K ♠ |  | K ♥ |                   |  6: SS_S_TO_BS_N     CLUBS   

In [15]:
gs = g.gs

In [9]:
def copy_from(other_gs):
    new_gs = VisibleGameState()
    new_gs.CopyFrom(other_gs)
    return new_gs
    
def merge_from(other_gs):
    new_gs = VisibleGameState()
    new_gs.MergeFrom(other_gs)
    return new_gs
    
def manual_merge_from(gs):
    new_gs = VisibleGameState(
        talon=gs.talon,
        suit_stack=gs.suit_stack,
        build_stacks=[
            gs.build_stacks[0],
            gs.build_stacks[1],
            gs.build_stacks[2],
            gs.build_stacks[3],
            gs.build_stacks[4],
            gs.build_stacks[5],
            gs.build_stacks[6],
        ],
        build_stacks_num_hidden=[
            gs.build_stacks_num_hidden[0],
            gs.build_stacks_num_hidden[1],
            gs.build_stacks_num_hidden[2],
            gs.build_stacks_num_hidden[3],
            gs.build_stacks_num_hidden[4],
            gs.build_stacks_num_hidden[5],
            gs.build_stacks_num_hidden[6],
        ]
    )
    return new_gs

print("Copy")
%timeit [copy_from(other) for other in states]
print("Merge")
%timeit [merge_from(other) for other in states]
print("Manual merge")
%timeit [manual_merge_from(other) for other in states]
# print("Copy")
# %timeit [copy_from(other) for other in states]
# print("Merge")
# %timeit [merge_from(other) for other in states]

Copy
248 µs ± 1.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Merge
202 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Manual merge
477 µs ± 2.94 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [59]:
print(len({base64.b64encode(gs.SerializeToString(deterministic=True)).decode("utf-8") for gs in states}))
print(len({base64.b64encode(gs.SerializeToString(deterministic=True)).decode("utf-8") for gs in [*states, *states]}))
print(len({base64.b64encode(gs.SerializeToString(deterministic=True)).decode("utf-8") for gs in itertools.chain(states, states)}))

print(len({base64.b64encode(gs.SerializeToString()).decode("utf-8") for gs in states}))
print(len({base64.b64encode(gs.SerializeToString()).decode("utf-8") for gs in [*states, *states]}))

print(len({gs.SerializeToString() for gs in itertools.chain(states, states)}))

101
101
101
101
101
101


In [60]:
print("b64 deterministic")
%timeit {base64.b64encode(gs.SerializeToString(deterministic=True)).decode("utf-8") for gs in states}
# %timeit {base64.b64encode(gs.SerializeToString(deterministic=True)).decode("utf-8") for gs in [*states, *states]}
%timeit {base64.b64encode(gs.SerializeToString(deterministic=True)).decode("utf-8") for gs in itertools.chain(states, states)}

print("b64 non deterministic")
%timeit {base64.b64encode(gs.SerializeToString()).decode("utf-8") for gs in states}
# %timeit {base64.b64encode(gs.SerializeToString()).decode("utf-8") for gs in [*states, *states]}
%timeit {base64.b64encode(gs.SerializeToString()).decode("utf-8") for gs in itertools.chain(states, states)}

print("just bytes")
%timeit {gs.SerializeToString() for gs in states}
%timeit {gs.SerializeToString() for gs in itertools.chain(states, states)}

print("md5 hex")
%timeit {hashlib.md5(g.gs.SerializeToString()).hexdigest() for gs in states}
%timeit {hashlib.md5(g.gs.SerializeToString()).hexdigest() for gs in itertools.chain(states, states)}

print("md5 int64")
%timeit {int.from_bytes(hashlib.md5(g.gs.SerializeToString()).digest()[:8], "big") for gs in states}
%timeit {int.from_bytes(hashlib.md5(g.gs.SerializeToString()).digest()[:8], "big") for gs in itertools.chain(states, states)}

print("md5 hex int64")
%timeit {int(hashlib.md5(g.gs.SerializeToString()).hexdigest()[:16], 16) for gs in states}
%timeit {int(hashlib.md5(g.gs.SerializeToString()).hexdigest()[:16], 16) for gs in itertools.chain(states, states)}

print("sha1 hex")
%timeit {hashlib.sha1(g.gs.SerializeToString()).hexdigest() for gs in states}
%timeit {hashlib.sha1(g.gs.SerializeToString()).hexdigest() for gs in itertools.chain(states, states)}

print("sha256 hex")
%timeit {hashlib.sha256(g.gs.SerializeToString()).hexdigest() for gs in states}
%timeit {hashlib.sha256(g.gs.SerializeToString()).hexdigest() for gs in itertools.chain(states, states)}

b64 deterministic
302 µs ± 4.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
598 µs ± 3.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
b64 non deterministic
272 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
558 µs ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
just bytes
193 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
389 µs ± 3.38 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
md5 hex
316 µs ± 1.62 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
635 µs ± 2.41 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
md5 int64
345 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
697 µs ± 6.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
md5 hex int64
364 µs ± 3.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
729 µs ± 3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
sha1 hex
320 µs ± 1.46 µs per loop (mean ± std. dev. of 

In [34]:
import hashlib

h = hashlib.md5(g.gs.SerializeToString())
h.hex

In [46]:
hdig = int.from_bytes(h.digest()[:8], byteorder="big")
print(hdig)
print(hdig.bit_length())

hdig2 = int(h.hexdigest()[:16], 16)
print(hdig2)
print(hdig2.bit_length())

1076927608774057088
60
1076927608774057088
60


In [51]:

len({gs.SerializeToString() for gs in states})

77

In [13]:
from solitaire_ai import vectorization
importlib.reload(vectorization)

len(vectorization.game_state_to_array(deal_game(is_random=False).gs))

475

In [15]:
[f"{i1} {i2}" for i1 in range(7) for i2 in range(7) if i1 != i2]

['0 1',
 '0 2',
 '0 3',
 '0 4',
 '0 5',
 '0 6',
 '1 0',
 '1 2',
 '1 3',
 '1 4',
 '1 5',
 '1 6',
 '2 0',
 '2 1',
 '2 3',
 '2 4',
 '2 5',
 '2 6',
 '3 0',
 '3 1',
 '3 2',
 '3 4',
 '3 5',
 '3 6',
 '4 0',
 '4 1',
 '4 2',
 '4 3',
 '4 5',
 '4 6',
 '5 0',
 '5 1',
 '5 2',
 '5 3',
 '5 4',
 '5 6',
 '6 0',
 '6 1',
 '6 2',
 '6 3',
 '6 4',
 '6 5']